In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error



In [2]:
train = pd.read_csv(r"C:\Walmart-Demand-Forecasting-Supply-Chain-Optimization--Python--Power-BI\Data\train.csv")
features = pd.read_csv(r"C:\Walmart-Demand-Forecasting-Supply-Chain-Optimization--Python--Power-BI\Data\features.csv")
stores = pd.read_csv(r"C:\Walmart-Demand-Forecasting-Supply-Chain-Optimization--Python--Power-BI\Data\stores.csv")

# Convert Date column to datetime
train["Date"] = pd.to_datetime(train["Date"])

train.head()


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [3]:
# Convert Date column to datetime
features["Date"] = pd.to_datetime(features["Date"])
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [4]:
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [5]:
df=train.merge(features, on=["Store","Date"],how="left")
df=df.merge(stores, on="Store",how="left")
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False,A,151315


In [6]:
# Fill MarkDown missing values with 0
markdown_cols= ["MarkDown1","MarkDown2","MarkDown3","MarkDown4","MarkDown5"]
for col in markdown_cols:
    if col in df.columns:
        df[col]=df[col].fillna(0)

# Fill remaining numeric missing values with median
numeric_cols= df.select_dtypes(include=["int64","float64"]).columns
for col in numeric_cols:
    df[col]= df[col].fillna(df[col].median())

df.isna().sum().sort_values(ascending=False).head(10)    

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday_x     0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
dtype: int64

In [7]:
#Creating a easy date features
df["Year"]= df["Date"].dt.year
df["Month"]=df["Date"].dt.month
df["Week"]=df["Date"].dt.isocalendar().week.astype(int)

df[["Date","Year","Month","Week"]].head()

,Date,Year,Month,Week
0,2010-02-05,2010,2,5
1,2010-02-12,2010,2,6
2,2010-02-19,2010,2,7
3,2010-02-26,2010,2,8
4,2010-03-05,2010,3,9


In [8]:
#Creating  lag features for “previous week sales” for each Store + Dept.

df=df.sort_values(["Store","Dept","Date"])

# lag_1 = previous week sales
df["lag_1"]= df.groupby(["Store","Dept"])["Weekly_Sales"].shift(1)

# lag_2 = 2 weeks before
df["lag_2"]=df.groupby(["Store","Dept"])["Weekly_Sales"].shift(2)

# if lag is missing (first weeks), fill with 0
df["lag_1"] = df["lag_1"].fillna(0)
df["lag_2"] = df["lag_2"].fillna(0)

df[["Store","Dept","Date","Weekly_Sales","lag_1","lag_2"]].head(10)

,Store,Dept,Date,Weekly_Sales,lag_1,lag_2
0,1,1,2010-02-05,24924.50,0.00,0.00
1,1,1,2010-02-12,46039.49,24924.50,0.00
2,1,1,2010-02-19,41595.55,46039.49,24924.50
3,1,1,2010-02-26,19403.54,41595.55,46039.49
4,1,1,2010-03-05,21827.90,19403.54,41595.55
5,1,1,2010-03-12,21043.39,21827.90,19403.54
6,1,1,2010-03-19,22136.64,21043.39,21827.90
7,1,1,2010-03-26,26229.21,22136.64,21043.39
8,1,1,2010-04-02,57258.43,26229.21,22136.64
9,1,1,2010-04-09,42960.91,57258.43,26229.21


In [9]:
# Convert Type into dummy variables (Type_A, Type_B, Type_C)
df=pd.get_dummies(df,columns=["Type"],drop_first=True)
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Unemployment,IsHoliday_y,Size,Year,Month,Week,lag_1,lag_2,Type_B,Type_C
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,...,8.106,False,151315,2010,2,5,0.00,0.00,False,False
1,1,1,2010-02-12,46039.49,True,38.51,2.548,0.0,0.0,0.0,...,8.106,True,151315,2010,2,6,24924.50,0.00,False,False
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.0,0.0,0.0,...,8.106,False,151315,2010,2,7,46039.49,24924.50,False,False
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.0,0.0,0.0,...,8.106,False,151315,2010,2,8,41595.55,46039.49,False,False
4,1,1,2010-03-05,21827.90,False,46.50,2.625,0.0,0.0,0.0,...,8.106,False,151315,2010,3,9,19403.54,41595.55,False,False


In [10]:
#Train/Validation split (last 16 weeks)
max_date=df["Date"].max()
cutoff_date= max_date - pd.Timedelta(weeks=16)

train_df= df[df["Date"] <= cutoff_date].copy()
val_df = df[df["Date"] > cutoff_date].copy()

train_df["Date"].min(), train_df["Date"].max(), val_df["Date"].min(), val_df["Date"].max()

(Timestamp('2010-02-05 00:00:00'),
 Timestamp('2012-07-06 00:00:00'),
 Timestamp('2012-07-13 00:00:00'),
 Timestamp('2012-10-26 00:00:00'))

In [11]:
#Pickinf the  features + Train model
target= "Weekly_Sales"

# Remove columns we should NOT use as inputs
drop_cols = ["Weekly_Sales", "Date"]

feature_cols=[c for c in df.columns if c not in drop_cols]

X_train= train_df[feature_cols]
y_train=train_df[target]

X_val= val_df[feature_cols]
y_val= val_df[target]

model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)

val_pred = model.predict(X_val)

mse = mean_squared_error(y_val, val_pred)
rmse = np.sqrt(mse)
rmse


np.float64(3245.8909953419684)

In [12]:
fact_val= val_df[["Store","Dept","Date","Weekly_Sales"]].copy()
fact_val["Predicted_Weekly_Sales"]= val_pred

fact_val.to_csv("fact_forecast_validation.csv", index=False)

fact_val.head()

,Store,Dept,Date,Weekly_Sales,Predicted_Weekly_Sales
127,1,1,2012-07-13,16566.18,17919.155601
128,1,1,2012-07-20,16348.06,16934.670994
129,1,1,2012-07-27,15731.18,17283.529136
130,1,1,2012-08-03,16628.31,16033.309147
131,1,1,2012-08-10,16119.92,17283.529136


In [13]:
import joblib

joblib.dump(model, "model_gbr.pkl")
print("Saved model: model_gbr.pkl")


Saved model: model_gbr.pkl


In [14]:
import json

with open("feature_cols.json", "w") as f:
    json.dump(feature_cols, f)

print("Saved feature columns: feature_cols.json")


Saved feature columns: feature_cols.json


In [20]:
tmp = df.sort_values(["Store", "Dept", "Date"]).copy()

lag_table = (
    tmp.groupby(["Store", "Dept"])["Weekly_Sales"]
      .agg(
          lag_1_start="last",
          lag_2_start=lambda s: s.iloc[-2] if len(s) > 1 else 0
      )
      .reset_index()
)

lag_table["lag_2_start"] = lag_table["lag_2_start"].fillna(0)

lag_table.to_csv("last_sales_lags.csv", index=False)

lag_table.head(), lag_table.columns



(   Store  Dept  lag_1_start  lag_2_start
 0      1     1     27390.81     24185.27
 1      1     2     43134.88     42354.72
 2      1     3      9350.90      8548.87
 3      1     4     36292.60     35549.19
 4      1     5     25846.94     20413.83,
 Index(['Store', 'Dept', 'lag_1_start', 'lag_2_start'], dtype='object'))